In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms

In [4]:
from torchvision.datasets import FashionMNIST

class NoisyFashionMNIST(FashionMNIST):
    def __init__(self, *args, noise_std=0.1, **kwargs):
        super(NoisyFashionMNIST, self).__init__(*args, **kwargs)
        self.noise_std = noise_std

    def __getitem__(self, index):
        img, label = super(NoisyFashionMNIST, self).__getitem__(index)
        img += self.noise_std * torch.randn_like(img)
        return img, label

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset_noisy = NoisyFashionMNIST(root='./data', train=True, download=True, transform=transform, noise_std=0.1)
test_dataset_noisy = NoisyFashionMNIST(root='./data', train=False, download=True, transform=transform, noise_std=0.1)

train_loader = DataLoader(dataset=train_dataset_noisy, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset_noisy, batch_size=64, shuffle=False)


# brak szumu w danych trenignowych
# train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
# train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)


# Definicja modelu
class CNNModel(nn.Module):
    def __init__(self, num_channels, kernel_size, pool_size):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, num_channels, kernel_size)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size)
        self.relu2 = nn.ReLU()

        self.pool = nn.MaxPool2d(pool_size)
        self.flatten = nn.Flatten()
        self.lazy_linear = nn.LazyLinear(10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.lazy_linear(x)
        return x


def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')


def test_model(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            # predicted to index przewidzianej klasy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Parametry do przebadania
num_channels_list = [16, 32, 64]
kernel_size_list = [3, 5]
pool_size_list = [2, 3]


model = CNNModel(num_channels=num_channels_list[0],
                 kernel_size=kernel_size_list[0],
                 pool_size=pool_size_list[0], )

# Definicja funkcji kosztu i optymalizatora
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trenowanie modelu
train_model(model, train_loader, criterion, optimizer)

# Testowanie modelu
test_model(model, test_loader, criterion)

Epoch 1/5, Loss: 0.19962810735200356
Epoch 2/5, Loss: 0.06352610135751802
Epoch 3/5, Loss: 0.04808249428105264
Epoch 4/5, Loss: 0.037718679769143865
Epoch 5/5, Loss: 0.03060157842002561
Test Accuracy: 98.69%
